In [1]:
import torch as tc
import numpy as np
import open3d as otd
from tqdm import tqdm
from src.star.star import STAR
otd_vector3d = otd.utility.Vector3dVector

genders = ['male', 'female']
nbetas = 300

In [2]:
star_parameters = tc.load("data/ss20_star_parameters.pt")
star_faces = STAR().faces
otd_faces = otd.utility.Vector3iVector(star_faces)

In [3]:
gender_bodies = {}
for gender in genders:
    gender_star = STAR(gender=gender, num_betas=nbetas)
    betas, poses, trans = star_parameters[gender]
    poses = np.zeros((betas.shape[0],72))
    poses[:,5] = 6 * np.pi/180
    poses[:,8] = -6 * np.pi/180
    poses = tc.cuda.FloatTensor(poses)
    bodies = gender_star(betas=betas, pose=poses, trans=trans)
    gender_bodies[gender] = bodies


In [4]:
nonsubdivided_gender_bodies = {}
nonsubdivided_gender_bodies['male'] = gender_bodies['male']
nonsubdivided_gender_bodies['female'] = gender_bodies['female']
nonsubdivided_bodies = {
    'vertices': nonsubdivided_gender_bodies,
    'faces': {
        'male': star_faces,
        'female': star_faces
    }
    
}
tc.save(nonsubdivided_bodies, "data/ss20_nonsubdivided_bodies.pt")

In [5]:
gender_star = STAR(gender=gender, num_betas=nbetas)
betas, poses, trans = star_parameters[gender]
body = gender_star(betas=betas, pose=poses, trans=trans)[4]
otd_vertices = otd_vector3d(body.detach().cpu())
otd_mesh = otd.geometry.TriangleMesh(otd_vertices, otd_faces)
otd.io.write_triangle_mesh("body_sample.obj", otd_mesh)

True

In [6]:
subdivided_gender_bodies = {}
subdivided_gender_faces = {}
for gender in genders:
    subdivided_gender_bodies[gender] = []
    subdivided_gender_faces[gender] = []
    for body in tqdm(gender_bodies[gender], desca=f"subdividing {gender} bodies"):
        otd_vertices = otd_vector3d(body.detach().cpu())
        otd_mesh = otd.geometry.TriangleMesh(otd_vertices, otd_faces)
        otd_mesh = otd_mesh.subdivide_loop(number_of_iterations=3)
        vertices = np.array(otd_mesh.vertices)
        subdivided_gender_bodies[gender].append(tc.FloatTensor(vertices))
    faces = np.array(otd_mesh.triangles)
    subdivided_gender_faces[gender] = tc.LongTensor(faces)


subdividing female bodies: 100%|██████████| 41/41 [02:02<00:00,  2.99s/it]


In [7]:
subdivided_gender_bodies['male'] = tc.stack(subdivided_gender_bodies['male'], dim=0)
subdivided_gender_bodies['female'] = tc.stack(subdivided_gender_bodies['female'], dim=0)
subdivided_bodies = {
    'vertices': subdivided_gender_bodies,
    'faces': subdivided_gender_faces
}
tc.save(subdivided_bodies, "data/ss20_subdivided_bodies.pt")